In [6]:
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import string
from preprocessing import preprocess_keypoints

We start with some defenitions

In [7]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# Constants for updating tracking aswell as when to save hand position
update_time = 5
save_time = 5

## Data collection
We use the cv2 library to collect camera fotage and then use mediapipe to get the landmarks on one hand.

To record data press the letter you are currently showing. Data can only be collected if the software is tracking your hand.

In [8]:
# Keep track of data
keypoint_data = []

# For webcam input:
cap = cv2.VideoCapture(1)
with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:

  i = 0

  # Open camera
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      continue

    # Convert BGR image to RGB and flip the image for selfie camera
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # Run mediapipe hands and save the results!
    results = hands.process(image)

    # Convert back to correct colors
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Save key press
    key = cv2.waitKey(update_time)
    i = i + update_time

    if results.multi_hand_landmarks:

        # Pick the first hand
        hand_landmarks = results.multi_hand_landmarks[0]

        # if save time has passed, save the keypoints based on keypress
        if not key == -1 and i >= save_time:
          i = 0

          # Is keypress in the alphabet?
          if chr(key) in list(string.ascii_lowercase):

            # If so save landmarks as an array with values [0,1] (except z axis)
            res = preprocess_keypoints(hand_landmarks)
            res.insert(0, chr(key))
            keypoint_data.append(res)
  
        # Draw on the image
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    
    # exit if space is pressed
    if key == ord(' '):
      break
            
    # Display the image!
    cv2.imshow('MediaPipe Hands', image)
    
cap.release()

## Saving the data
After we have collected all the data we need, we can save it into a csv file with pandas!

In [9]:
# Save the data to a csv file
df = pd.DataFrame(np.array(keypoint_data))
df.to_csv('data.csv', index = False)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,a,0.950764520849804,0.6417859168943352,0.0,0.83713399318848,0.6284014988976199,-0.018810274436540163,0.7662351175700984,0.5163007646771525,-0.023713984709434144,...,-0.029889471614473744,0.9661612980680846,0.3259206188036239,-0.05231681349686658,0.9565646883764578,0.4108489212959662,-0.04068884171857279,0.978882918458999,0.4350473232280454,-0.02167949191818614
1,a,0.953936997698104,0.6564206472929742,0.0,0.8462258984177313,0.667826893015441,-0.035542203587109814,0.7701451841646806,0.5521304276210391,-0.043699686649416085,...,-0.017873136496193407,0.9672263904248711,0.35499974535533485,-0.04512985004411547,0.9631982635833811,0.4436075242069195,-0.03614679827156683,0.9844323404536468,0.4636945148009726,-0.017158629517706064
2,a,0.9527207560910744,0.6563568575159713,0.0,0.8429681704058657,0.6653577596707893,-0.03103226090488621,0.7687039603204814,0.5515818754818361,-0.03805677215230527,...,-0.020251425283030238,0.965818900105203,0.3550167699248286,-0.046689752108532635,0.9600699762601301,0.44247115176831675,-0.03697539120011366,0.9823489487120628,0.4643287541469835,-0.017594370303124936
3,a,0.9526128183185797,0.656659720486357,0.0,0.8438174150969431,0.6662704304745074,-0.0323668762656943,0.7687742679051285,0.5513059578768369,-0.039399063967528465,...,-0.019865503774251076,0.9660724322104915,0.3545698314467687,-0.047071919775441846,0.9607358259830384,0.44262490503946345,-0.037663000927347194,0.9826914057157752,0.4654384186083795,-0.018333040111284723
4,a,0.9523731395164919,0.6548508823771029,0.0,0.8460813518215848,0.6703589220445302,-0.03993449866950677,0.7698222187618396,0.5519360584662958,-0.04824125186523472,...,-0.01412351454630196,0.9688928812711428,0.35284416875822266,-0.04263958473315342,0.9644267399559332,0.4428907505814873,-0.03393987451506962,0.9837459430761628,0.4678725580291841,-0.014516818809612411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15323,g,0.6237660632453513,1.0,0.0,0.6162153993893648,0.9556866743600728,-0.03319059493711618,0.6400647482786056,0.9091647648726392,-0.051560150690229775,...,-0.031136176286684327,0.7338897487360457,0.96682089232996,-0.048610222082431334,0.7018132925149551,0.983072445260195,-0.04598488665265533,0.6803640213589819,0.9820834282390071,-0.03985719666832186
15324,g,0.6234343309982603,0.9999999999999999,0.0,0.6155897623371828,0.9552162362662298,-0.03251277962747347,0.6388733943304896,0.9085140846479999,-0.05061660343376199,...,-0.031587264319584106,0.7333846825306606,0.9649105845437027,-0.0492081079488784,0.7014559681577068,0.9814960353862563,-0.04691002994971202,0.6794612903968944,0.9810501445872041,-0.041105215783073995
15325,g,0.6215343522580493,1.0,0.0,0.6138994436091848,0.9559501142971435,-0.03158360063104972,0.6376567241076002,0.9091851538437794,-0.04936649798144895,...,-0.03177966457257837,0.7307098318129266,0.9669501321149,-0.04922794767175878,0.6993314451530558,0.9830828558300075,-0.04684075649927763,0.6767986962421347,0.9814727956737483,-0.041042978530356315
15326,g,0.6206172349797326,0.9999999999999999,0.0,0.6125545053306167,0.9558512846767386,-0.03242856304979096,0.6359976892450179,0.9098832472830194,-0.050963591274790485,...,-0.032444689072443696,0.7305952384901823,0.9682023313987947,-0.049399777856800414,0.6983575274270704,0.9841983443511911,-0.0471813895401719,0.6757752743833756,0.9820242967285815,-0.04182518314811384
